**3. Density Operator master equations**

**3.1. Introduction to Lindblad master equation**

The Gorini-Kossakowski-Sudarshan-Lindblad (GKSL) master equation, often known as the *Lindblad* master equation:

$$\dot{\rho}(t) = -\frac{i}{\hbar} [H, \rho(t)] + \sum_k \gamma_k \left(L_k\rho(t)L_k^\dagger - \frac{1}{2}\Bigl\{L_k^\dagger L_k, \rho(t)\Bigr\}\right)$$

where $\{L_k\}$ are the *Lindblad* operators representing some non-unitary processes like relaxation or decoherence that occur at some rates $\{\gamma_k\}$.

Like the Hamiltonian generates coherent dynamics, the Lindblad operators generate incoherent transitions in the space of states. Unlike the Hamiltonian, they do not need to be hermitian. For example, a decay transition from some excited state $|e\rangle$ to some ground state $|g\rangle$ is mediated by the Lindblad operator

$$L_\downarrow = |g\rangle\langle e|$$

Indeed, when we apply $L_\downarrow$ to $|e\rangle$ we obtain $|g\rangle = L_\downarrow|e\rangle$. Note that $L_\downarrow^\dagger = |e\rangle\langle g| \neq L_\downarrow$.

This Lindblad equation is used to approxiamte the evolution of the density operator of a system $\text{S}$ with Hamiltonian $H$ that is weakly coupled to a Markovian (memory-less) environment, and is a general form for a **completely positive and trace-preserving (CPTP) Markovian** and **time-homogeneous map** for the evolution of the system's density operator $\rho$.

**3.2 The Liouville superoperator**

When solving the Lindblad equation, it is convenient to express the master equation in a vector notation,

$$\dot{\boldsymbol{\rho}} = \mathcal{L}\boldsymbol{\rho}$$

known as a *superoperator* or *Liouville* form, where $\boldsymbol{\rho} = \text{vec}(\rho)$ is the *vectorized* form of $\rho$, and $\mathcal{L}$ is the superoperator associated with the generator $\dot{\rho}$.

In [8]:
import numpy as np

psi = np.array([1, 2j, 0, -2.0j])
rho = np.outer(psi, psi.conj())

rho /= np.trace(rho)
d = len(psi)

vec_rho = np.reshape(rho, (d**2, 1))

print(f'vectorized rho = \n{vec_rho}\n')

vectorized rho = 
[[ 0.11111111+0.j        ]
 [ 0.        -0.22222222j]
 [ 0.        +0.j        ]
 [ 0.        +0.22222222j]
 [ 0.        +0.22222222j]
 [ 0.44444444+0.j        ]
 [ 0.        +0.j        ]
 [-0.44444444+0.j        ]
 [ 0.        +0.j        ]
 [ 0.        +0.j        ]
 [ 0.        +0.j        ]
 [ 0.        +0.j        ]
 [-0.        -0.22222222j]
 [-0.44444444+0.j        ]
 [ 0.        +0.j        ]
 [ 0.44444444+0.j        ]]



**3.2.1. Constructing the Liouville superoperator**

Two common ways to construct the superoperator $\mathcal{L}$ are to either follow an index prescription for the superoperator tensor $\rho_{ab} = \sum_{cd}\mathcal{L}_{abcd}\rho_{cd}$, or to use the following linear algebra idenity for the column-ordered form of $\text{vec}(\rho)$:

$$\text{vec}(AXB) = (B^\text{T} \otimes A)\text{vec}(X)$$

Thus, the Lindblad equation:

$$ \mathbf{1}\dot{\rho}\mathbf{1} = -\frac{i}{\hbar}\left(H\rho\mathbf{1} - \mathbf{1}\rho H \right) + \sum_k \gamma_k \left(L_k\rho L_k^\dagger - \frac{1}{2} \left(L_k^\dagger L_k \rho \mathbf{1} + \mathbf{1}\rho L_k^\dagger L_k \right) \right) $$

Now we obtain the Liouville superoperator:

$$ \mathcal{L} = -\frac{i}{\hbar}\left(\mathbf{1} \otimes H - H^\text{T} \otimes \mathbf{1} \right) + \sum_k \gamma_k \left(L_k^* \otimes L_k - \frac{1}{2} \left(\mathbf{1} \otimes L_k^\dagger L_k + L_k^\text{T} L_k^* \otimes \mathbf{1} \right) \right) $$

In [15]:
import numpy as np

'''
Constructs the Liouville superoperator
from the Hamiltonian and the set of Lindblad operators.
'''
def Liouvillian(H, Ls, hbar = 1):
    d = len(H)
    superH = -1j/hbar * (np.kron(np.eye(d), H) - np.kron(H.T, np.eye(d)))
    superL = sum([np.kron(L.conj(), L)
                  - 1/2 * (np.kron(np.eye(d), L.conj().T @ L) +
                           np.kron(L.T @ L.conj(), np.eye(d))
                           ) for L in Ls])
    return superH + superL

H = np.array([[0, 1], [1, 1]]) # Hamiltonian
Ls = [np.array([[0, 1], [0, 0]])] # Lindblad operators
superop = Liouvillian(H, Ls)

print(f'Hamiltonian = \n{H}\n')
print(f'Lindblad operators = \n{Ls}\n')
print(f'Liouvillian superoperator = \n{superop}\n')

Hamiltonian = 
[[0 1]
 [1 1]]

Lindblad operators = 
[array([[0, 1],
       [0, 0]])]

Liouvillian superoperator = 
[[ 0. +0.j  0. -1.j  0. +1.j  1. +0.j]
 [ 0. -1.j -0.5-1.j  0. +0.j  0. +1.j]
 [ 0. +1.j  0. +0.j -0.5+1.j  0. -1.j]
 [ 0. +0.j  0. +1.j  0. -1.j -1. +0.j]]



**3.3 Steady-state solution**

**3.3.1. Using the null space of Liouville superoperator**


The steady state solution ($\dot{\rho}=0$) which is often measured directly in experiments. To find any steady state solutions we solve for the *null space* of $\mathcal{L}$, which is the subspace of all vectors $\boldsymbol{\rho}$ that satisfy the equation.

$$\mathcal{L}\boldsymbol{\rho} = 0 $$

If there is a unique solution, solving for the null space will provide the corresponding steady dtate density matrix vector $\boldsymbol{\rho}(\infty)$ up to a constant factor.

If there are multiple solutions, solving for the null space will give linearly independent vectors. In such case, the steady state depends on the inital state of the system. For example, let us consider a two-level Hamiltonian $H$ with energy splitting $\Delta$ and coupling $\Omega$, and Lindblad operators $L_\downarrow$ and $L_0$ associated with spontaneous relaxation and depasing, respectively,

$$H=\hbar \begin{pmatrix} 0 & \Omega \\ \Omega & \Delta \end{pmatrix}, \quad L_\downarrow = \sqrt{\gamma_\downarrow} \begin{pmatrix} 0 & 1 \\ 0 & 0 \end{pmatrix}, \quad L_0 = \sqrt{\gamma_0} \begin{pmatrix} 1 & 0 \\ 0 & 1 \end{pmatrix}$$

where $\gamma_\downarrow$ and $\gamma_0$ are the rates associated with relaxation and dephasing. In the following script, we construct $\mathcal{L}$ in python and solve for its null space for the case of $\text{(i)}$ relaxation and no-driving limit $\Omega, \gamma_0 = 0$, and $\text{(ii)}$ dephasing and driving, with no relaxation, $\gamma_\downarrow = 0$.

In [56]:
from scipy.linalg import null_space

# two-level Hamiltonian from delta and omega parameters
def H_tls(omega, delta):
    return np.array([[0, omega], [omega, delta]])

# Lindblad operators for spontaneous relaxation and dephasing
def Ls_tls(g_relax, g_deph):
    return [np.array([[0, 1], [0, 0]]) * np.sqrt(g_relax),
            np.array([[1, 0], [0, 1]]) * np.sqrt(g_deph)]

# ------ (i) relaxation with no-driving ------
omega, delta, g_relax, g_deph = 0, 1, 1, 0
superop = Liouvillian(H_tls(omega, delta), Ls_tls(g_relax, g_deph))
null = null_space(superop)
print(f'(i) The steady-state space is a linear subspace of dimension {len(null.T)}.\n')
print(f'null = \n{null}\n')
rho_ss = np.reshape(null, (2, 2))
rho_ss /= np.trace(rho_ss)
print(f'rho_ss = \n{rho_ss}\n')

# ------ (ii) dephasing with driving ------
omega, delta, g_relax, g_deph = 1, 1, 0, 1
superop = Liouvillian(H_tls(omega, delta), Ls_tls(g_relax, g_deph))
null = null_space(superop)
print(f'(ii) The steady-state space is a linear subspace of dimension {len(null.T)}.\n')
print(f'null = \n{null}\n')

(i) The steady-state space is a linear subspace of dimension 1.

null = 
[[-1.-0.j]
 [ 0.-0.j]
 [-0.-0.j]
 [ 0.-0.j]]

rho_ss = 
[[ 1.+0.j -0.-0.j]
 [-0.+0.j -0.-0.j]]

(ii) The steady-state space is a linear subspace of dimension 2.

null = 
[[ 0.61083942-0.j         -0.47631419-0.j        ]
 [ 0.13501687-0.10663722j  0.59304055-0.13675473j]
 [ 0.13501687-0.10663722j  0.59304055-0.13675473j]
 [ 0.74585629-0.10663722j  0.11672636-0.13675473j]]



In the limit of relaxation and no-driving, there is a unique steady state $\boldsymbol{\rho}(\infty) = (1, 0, 0, 0)^\text{T}$, which is the ground state of the system, as expected for a two-system undergoing spontaneous relaxation with no driving field. 

Instead, for the case of dephasing and driving, the null function returns two vectors that spand the two-dimensional linear subspace associated with the null space of $\mathcal{L}$. In this case, the specific steady state depends on the choice of inital state.

**3.3.2. Algebraic solution**

The steady state solution $\boldsymbol{\rho}(\infty)$ for both linear and non-linear generators can be obtaind by the Lindblad equation for $\dot{\rho} = 0$ algebraically (or symbolically). In python, this can be done using the solve method of the SymPy library.

In [69]:
from sympy import *

# real variables
a, Gamma, Omega, Delta = symbols('a Gamma Omega Delta', real=True)

# complex variables
b = symbols('b')

# general TLS density operator
rho = Matrix([[a, b], [b.conjugate(), 1 - a]])

# Hamiltonian
H_s = Matrix([[0, Omega], [Omega, Delta]])

# Lindblad operators
L = Matrix([[0, 1], [0, 0]])

# Generator
rho_dot = (-1j) * (H_s*rho - rho*H_s) + Gamma * (L*rho*L.H - 1/2 * (L.H*L*rho + rho*L.H*L))

# Steady-state solution for Omega = 0
rho_dot = rho_dot.subs(Omega, 0)
sol = solve(flatten(rho_dot), [a, b], dict=True)

# steady state
rho_ss = rho.subs(sol[0])
print(f'rho_ss = \n{rho_ss}\n')

rho_ss = 
Matrix([[1.00000000000000, 0], [0, 0]])



**3.4 Solving the dynamics of the system**

